# Simple Neural Network

# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [1]:
def sigmoid(x) :
    return 1/(1+np.exp(-x))

In [2]:
def derivativeSigmoid(x) :
    return (np.exp(-x))/((1+np.exp(-x))**2)

In [3]:
class Perceptron :
    def __init__(self):
        self.weights=np.random.rand(3,1)
    def forward(self,inputs,expected outputs,its,lrate) :
        y=np.dot(inputs,self.weights)
        activation=sigmoid(y)
        bias=np.random
        

# 